In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

## Part 1: Import the Housing data and do feature transformations

In [12]:
df= pd.read_csv('house_price_full.csv')
df.head()

,bedrooms,sqft_living,price
0,3,1340,313000
1,5,3650,2384000
2,3,1930,342000
3,3,2000,420000
4,4,1940,550000


In [13]:
X = df.copy()
# Remove target
Y = X.pop('price')

# perform a scaler transform of the input data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# perform log transformation of target variable (For Sandeep: Is this needed?)
Y = np.log(Y)

In [14]:
df_scaled = pd.DataFrame(X)
df_scaled

,0,1
0,-0.433198,-0.753258
1,1.675735,1.457330
2,-0.433198,-0.188649
3,-0.433198,-0.121661
4,0.621269,-0.179079
...,...,...
494,0.621269,0.873582
495,1.675735,2.299459
496,-0.433198,-0.724549
497,-0.433198,-0.179079


In [15]:
Y

0      12.653958
1      14.684290
2      12.742566
3      12.948010
4      13.217674
         ...    
494    13.380102
495    13.764217
496    12.128111
497    12.721886
498    12.254863
Name: price, Length: 499, dtype: float64

## Part 2: Create Model Using `keras`

![](multiple_neurons.png)

In [16]:
from tensorflow import keras

In [17]:
model = keras.Sequential(
    [
        keras.layers.Dense(
            2, activation="sigmoid", input_shape=(X.shape[-1],)
        ),
        keras.layers.Dense(1, activation="linear")
    ]
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 2)                 6         
                                                                 
 dense_3 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9 (36.00 Byte)
Trainable params: 9 (36.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


```python
def random_init_params():
    w1 = tf.Variable(tf.random.uniform((2, 2)))
    b1 = tf.Variable(tf.random.uniform((1, 2)))
    w2 = tf.Variable(tf.random.uniform((2, 1)))
    b2 = tf.Variable(tf.random.uniform((1, 1)))
    return w1,b1,w2,b2


def forward_prop(x, w1, b1, w2, b2):
    z1 = tf.matmul(x,w1) + b1
    h1 = tf.math.sigmoid(z1)
    z2 = tf.matmul(h1,w2) + b2
    h2 = z2
    return h2
```

In [18]:
model.compile(
    optimizer=keras.optimizers.SGD(), loss="mean_squared_error"
)

```python
def train(x, y, w1, b1, w2, b2):
    y_true = y
    with tf.GradientTape() as g:
        y_pred = forward_prop(x, w1, b1, w2, b2)

        # loss
        loss = 0.5*(y_true - y_pred)** 2
    
    #Gradient calculation  
    print("**************************************************")
    print("GRADIENTS")
    print("**************************************************")
    gw1, gb1, gw2, gb2 = g.gradient(loss, [w1, b1, w2, b2])
    print(" the gradient for 1st layer weights are:\n",gw1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer weights are:\n",gw2.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 1st layer bias are:\n",gb1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer bias are:\n",gb2.numpy())
    print("--------------------------------------------------")

    # Gradient descent:
    lr=0.2
    w1.assign_sub(lr*gw1)
    b1.assign_sub(lr*gb1) 
    w2.assign_sub(lr*gw2)
    b2.assign_sub(lr*gb2)
    print("**************************************************")
    print("NEW UPDATES")
    print("**************************************************")
    print(" the updated 1st layer weights are:\n",w1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer weights are:\n",w2.numpy())
    print("--------------------------------------------------")
    print(" the updated 1st layer bias are:\n",b1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer bias are:\n",b2.numpy())


    return w1, b1, w2, b2,loss

```

In [19]:
model.fit(X,Y.values,epochs=10,batch_size=32)

Epoch 1/10


16/16 [==============================] - 1s 5ms/step - loss: 111.4731
Epoch 2/10
16/16 [==============================] - 0s 5ms/step - loss: 32.0273
Epoch 3/10
16/16 [==============================] - 0s 7ms/step - loss: 6.4743
Epoch 4/10
16/16 [==============================] - 1s 49ms/step - loss: 1.3505
Epoch 5/10
16/16 [==============================] - 0s 3ms/step - loss: 0.4504
Epoch 6/10
16/16 [==============================] - 0s 3ms/step - loss: 0.2717
Epoch 7/10
16/16 [==============================] - 0s 3ms/step - loss: 0.2170
Epoch 8/10
16/16 [==============================] - 0s 3ms/step - loss: 0.1933
Epoch 9/10
16/16 [==============================] - 0s 3ms/step - loss: 0.1796
Epoch 10/10
16/16 [==============================] - 0s 3ms/step - loss: 0.1702


In [20]:
model.predict(X)[:,0]

16/16 [==============================] - 0s 3ms/step


array([12.918899 , 13.371866 , 13.059525 , 13.074423 , 13.056641 ,
       12.807037 , 12.929087 , 13.21512  , 13.158388 , 12.951017 ,
       13.010309 , 13.251373 , 13.139977 , 12.850607 , 13.253237 ,
       12.881327 , 12.977032 , 13.281887 , 12.926731 , 12.87582  ,
       13.122843 , 12.931965 , 13.019533 , 13.218692 , 12.972275 ,
       13.093056 , 13.145569 , 13.114955 , 13.236519 , 13.188223 ,
       12.961715 , 13.241293 , 13.0279455, 13.234451 , 13.355352 ,
       13.285514 , 12.784373 , 13.042053 , 13.065953 , 13.151102 ,
       12.979461 , 12.940183 , 12.798605 , 13.024098 , 12.894661 ,
       12.870279 , 13.068082 , 13.059525 , 12.900324 , 13.31181  ,
       13.21512  , 13.238161 , 13.206583 , 13.030889 , 13.108124 ,
       13.051939 , 13.112807 , 13.373259 , 12.913634 , 12.993294 ,
       13.31181  , 13.201647 , 13.218692 , 13.177898 , 13.251373 ,
       12.812614 , 12.996238 , 12.827591 , 13.185545 , 13.115052 ,
       13.181805 , 13.228926 , 13.131874 , 13.053028 , 12.9162